# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [1]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [2]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [3]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [4]:
example_document = index.document(index.maximum_document()-2)
len(set(example_document[1]))

120

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [5]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [6]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['preston', 'robert', 'tisch', 'joins', 'brother', 'cbs', 'board', 'businessman', 'preston', 'robert', 'tisch', 'elected', 'board', 'cbs', 'joins', 'brother', 'cbs', 'chief', 'executive', 'laurence', 'tisch', 'communications', 'giant', 'announced', 'wednesday', 'preston', 'tisch', 'better', 'known', 'bob', 'stepped', 'march', 'serving', '19', 'months', 'postmaster', 'general', 'speculated', 'widely', 'eventually', 'join', 'cbs', 'board', 'tisch', 'brothers', 'considerable', 'economic', 'ties', 'serve', 'co', 'chief', 'executives', 'loews', 'corp', 'tobacco', 'real', 'estate', 'insurance', 'hotel', 'conglomerate', 'cbss', 'biggest', 'shareholder', 'addition', 'preston', 'president', 'laurence', 'chairman', 'loews', 'prestons', 'election', 'cbs', 'board', 'regular', 'board', 'meeting', 'announced', 'william', 's', 'paley', 'cbs', 'founder', 'chairman', 'statement', 'paley', 'quoted', 'saying', 'new', 'board', 'member', 'broad', 'experience', 'business', 'public', 'affairs', 'valuable', '

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [7]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [8]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP880914-0270 has 0 word matches with query: "university  massachusetts".
Document AP880914-0270 and query "university  massachusetts" have a 0.0% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [9]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [10]:
# libraries
import time
import numpy as np
import math

In [11]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

start_time = time.time()

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

#______________________we added a dict to keep the ext_doc_______________________
ext_dic = {}

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
    
    #--------------------------------------
    ext_dic[int_doc_id] = ext_doc_id
    #--------------------------------------
    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 48.86119294166565 seconds.


In [13]:
def get_tf(int_document_id, query_term_id):
    #Returns term frequency (tf_t) for a document
    #https://docs.quantifiedcode.com/python-anti-patterns/correctness/not_using_get_to_return_a_default_value_from_a_dictionary.html
    
    return float(inverted_index.get(query_term_id, 0).get(int_document_id, 0))

def collection_freq(query_term_id):
    #Returns collection frequency
    return collection_frequencies.get(query_term_id, 0)

def calc_p_wc(query_term_id):
    #Returns background probability p(w|C)
    tf_wC = collection_freq(query_term_id)
    C = total_terms
    return (tf_wC/C)

In [14]:
import collections
from collections import Counter
def run_retrieval(model_name, score_fn):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = 'models/{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)

    data = collections.defaultdict(list)

    # TODO: fill the data dictionary. 
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    
    for query_id, _ in queries.items():
        
        score_per_doc = Counter()
        '''
        We do not go through all documents, we just go through the all documents relate to query term.
        For example if we have 5 docs and Q1 = w1 w2 and w1 is inside the doc1 & doc2, w2 is inside doc3
        , then we go through doc1, doc2, doc3
        '''        
        #------------------------docs_to_check = sum of all documents for q:-------------------------
        
        docs_to_check = set([inverted_index[term_id].keys() for term_id in tokenized_queries[query_id]][0])
        
        '''
        We check the score for every term of a query for every doc related to that query and assign some score
        to them.
        In tf-idf when the term is not inside the that doc, we return 0.
        In the smoothing methods, we assign some probability to the unseen terms.
        '''
        #--------------------------find the score(query, doc)-----------------------------------------
        for query_term_id in tokenized_queries[query_id]:
            for int_doc_id in docs_to_check:
                document_term_freq = get_tf(int_doc_id, query_term_id)
                score_per_doc[int_doc_id] += score_fn(int_doc_id, query_term_id, document_term_freq)
                
        #------------------------make data set to write in run file------------------------------------        
        for int_doc_id in score_per_doc:
            data[query_id].append((score_per_doc[int_doc_id], ext_dic[int_doc_id]))

    print('Retrieval took: ', time.time() - retrieval_start_time, 'seconds.')
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

#### TF - IDF

In [15]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    tf = np.log10(1 + document_term_freq)    
    idf = np.log10(num_documents/len(inverted_index[query_term_id]))

    return (tf * idf)

# combining the two functions above: 
run_retrieval('tfidf', tfidf)

# TODO implement the rest of the retrieval functions 

# TODO implement tools to help you with the analysis of the results.

#### BM25

In [16]:
def bm25(int_document_id, query_term_id, document_term_freq):
    
    K1 = 1.2
    B = 0.75
    
    tf = document_term_freq
    normalize_l =  document_lengths[int_document_id]/avg_doc_length
    idf = np.log10(num_documents/len(inverted_index[query_term_id]))
    w = ((K1+1)*tf)/ (K1*( (1-B) + B*(normalize_l) ) + tf)   
    score = w*idf
    
    return score

run_retrieval('bm25', bm25)

#### Jelinek-Mercer at 𝛌 = [0.1, 0.5, 0.9]

In [17]:
def jelinek_mercer(int_document_id, query_term_id, document_term_freq):
    
    tf_wd = document_term_freq
    d = document_lengths[int_document_id]
    p_wC = calc_p_wc(query_term_id)
    
    return tf_wd, d, p_wC

In [18]:
def jelinek_mercer_01(int_document_id, query_term_id, document_term_freq, lmbda = 0.1):
    
    tf_wd, d, p_wC = jelinek_mercer(int_document_id, query_term_id, document_term_freq)
    #p(w|d) = seen smoothed pbt + unseen smoothed pbt
    pbt_lambda = (1 - lmbda)*(p_wC) + lmbda*(tf_wd/d) 
    
    return np.log10(pbt_lambda)

def jelinek_mercer_05(int_document_id, query_term_id, document_term_freq, lmbda = 0.5):
    
    tf_wd, d, p_wC = jelinek_mercer(int_document_id, query_term_id, document_term_freq)
    #p(w|d) = seen smoothed pbt + unseen smoothed pbt
    pbt_lambda = (1 - lmbda)*(p_wC) + lmbda*(tf_wd/d) 
    
    return np.log10(pbt_lambda)

def jelinek_mercer_09(int_document_id, query_term_id, document_term_freq, lmbda = 0.9):

    tf_wd, d, p_wC = jelinek_mercer(int_document_id, query_term_id, document_term_freq)
    #p(w|d) = seen smoothed pbt + unseen smoothed pbt
    pbt_lambda = (1 - lmbda)*(p_wC) + lmbda*(tf_wd/d) 
    
    return np.log10(pbt_lambda)


run_retrieval('jelinek_mercer 0.1', jelinek_mercer_01)
run_retrieval('jelinek_mercer 0.5', jelinek_mercer_05)
run_retrieval('jelinek_mercer 0.9', jelinek_mercer_09)

#### Absolute Discounting at 𝛅 = [0.1, 0.5, 0.9]

In [19]:
#formula from : https://dl.acm.org/citation.cfm?id=384019
def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta):
    
    tf_wd = document_term_freq
    d = document_lengths[int_document_id]
    #d_u = number of unique words in d
    d_u = unique_terms_per_document[int_document_id]
    p_wC = calc_p_wc(query_term_id)
    
    #seen words probability
    seen_words = max((tf_wd - delta), 0)/d
    
    #unseen words probability
    unseen_words = delta*(d_u/d)*p_wC

    return seen_words, unseen_words

In [20]:
def absolute_discounting_01(int_document_id, query_term_id, document_term_freq):
    
    seen_words, unseen_words = absolute_discounting(int_document_id, query_term_id, document_term_freq, delta = 0.1)
    return (seen_words + unseen_words)

def absolute_discounting_05(int_document_id, query_term_id, document_term_freq):
    
    seen_words, unseen_words = absolute_discounting(int_document_id, query_term_id, document_term_freq, delta = 0.5)
    return (seen_words + unseen_words)

def absolute_discounting_09(int_document_id, query_term_id, document_term_freq):
    
    seen_words, unseen_words = absolute_discounting(int_document_id, query_term_id, document_term_freq, delta = 0.9)
    return (seen_words + unseen_words)


run_retrieval('absolute_discounting 0.1', absolute_discounting_01)
run_retrieval('absolute_discounting 0.5', absolute_discounting_05)
run_retrieval('absolute_discounting 0.9', absolute_discounting_09)

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [39]:
from operator import itemgetter
from pprint import pprint
import copy
import gensim
import logging
import pyndri
import pyndri.compat
import sys
import Cython
import multiprocessing
from sklearn.metrics.pairwise import cosine_similarity

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [22]:
#first obtain an initial top-1000 ranking for each query using TF-IDF in Task 1
docx = {}
keys = inverted_index.keys()
start_time = time.time()

print('Computing took: ')

for k in keys:
    docx[k] = inverted_index[k].keys()

scores = {}
top1000_tfidf = {}

#create datasets to store scores and document ids separately
for query_id, term_idx in tokenized_queries.items():
    scores[query_id] = []
    docx_set = set()

    for term_id in term_idx:
        docx_set = docx_set | set(docx[term_id])

#compute tfids scores
    for d_id in docx_set:
        result = 0
        for term_id in term_idx:
            result += tfidf(d_id, term_id, len(inverted_index.get(term_id, 0)))
        scores[query_id].append((result, index.document(d_id)[0], d_id))  

print(time.time() - start_time, 'seconds.')

Computing took: 
166.08451437950134 seconds.


In [23]:
#store top 1000 tfidf rank queries
for query_id, term_idx in tokenized_queries.items():
    top1000_tfidf[query_id] = sorted(scores[query_id], key = itemgetter(0), reverse = True)
    del top1000_tfidf[query_id][1000:]

#pprint(top1000_tfidf)

## LSMs

In [24]:
#Use pyndri and gensim to create LSM Models
document = index.document(index.document_base())

#dictionary
dictionary = pyndri.extract_dictionary(index)

#indri to get sentences
sentences = pyndri.compat.IndriSentences(index, dictionary)

#building corpus from sentences
dictionary_corpus = gensim.corpora.Dictionary(sentences)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(73866 unique tokens: ['aiding', 'caused', 'ame', 'batang', 'zamudio']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(101162 unique tokens: ['screeening', 'caused', 'polik', 'zamudio', 'casino']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(121185 unique tokens: ['screeening', 'caused', 'polik', 'zamudio', 'neighborhing']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(138774 unique tokens: ['screeening', 'caused', 'dumpsite', 'polik', 'zamudio']...)
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary(153153 unique tokens: ['screeening', 'caused', 'dumpsite', 'polik', 'zamudio']...)
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary(165943 unique tokens: ['screeening', 'caused', 'dumpsite', 'polik', 'zamudio']...)
INFO:gensim.corp

In [31]:
list(dictionary_corpus.token2id.items())[:5]

[('screeening', 99389),
 ('caused', 896),
 ('7015', 187357),
 ('novetzke', 95617),
 ('zamudio', 36900)]

In [32]:
## convert tokenized documents to vectors [DOCUMENT BOW]
if os.path.exists('models/tharangni.mm'):
    corpus = gensim.corpora.MmCorpus('models/tharangni.mm')
    print('Using already saved BOW')
else:
    corpus = [dictionary_corpus.doc2bow(stuff) for stuff in sentences]
    gensim.corpora.MmCorpus.serialize('models/tharangni.mm', corpus) ##<-- saves bow to disk
    print('BOW saved to disk')

INFO:gensim.corpora.indexedcorpus:loaded corpus index from models/tharangni.mm.index
INFO:gensim.matutils:initializing corpus reader from models/tharangni.mm
INFO:gensim.matutils:accepted corpus with 164597 documents, 267318 features, 29573706 non-zero entries


Using already saved BOW


In [40]:
print(corpus)

MmCorpus(164597 documents, 267318 features, 29573706 non-zero entries)


In [33]:
#representation for q (query) [QUERY BOW]
query_bow = []

#query id and term id in tokenized queries
for q_id, t_id in tokenized_queries.items():
    word_at_term = [id2token[t] for t in t_id]
    query_bow.append(dictionary_corpus.doc2bow(word_at_term))

#### LSI

In [34]:
#train transformation model for LSI
lsi_train = gensim.models.LsiModel(corpus=corpus, id2word=dictionary_corpus, num_topics=100)

INFO:gensim.models.lsimodel:using serial LSI version on this node
INFO:gensim.models.lsimodel:updating model with new documents
INFO:gensim.models.lsimodel:preparing a new chunk of documents
INFO:gensim.models.lsimodel:using 100 extra samples and 2 power iterations
INFO:gensim.models.lsimodel:1st phase: constructing (267318, 200) action matrix
INFO:gensim.models.lsimodel:orthonormalizing (267318, 200) action matrix
INFO:gensim.models.lsimodel:2nd phase: running dense svd on (200, 20000) matrix
INFO:gensim.models.lsimodel:computing the final decomposition
INFO:gensim.models.lsimodel:keeping 100 factors (discarding 16.353% of energy spectrum)
INFO:gensim.models.lsimodel:processed documents up to #20000
INFO:gensim.models.lsimodel:topic #0(876.656): 0.235*"percent" + 0.199*"1" + 0.199*"new" + 0.193*"million" + 0.163*"000" + 0.145*"government" + 0.144*"two" + 0.141*"people" + 0.129*"president" + 0.127*"2"
INFO:gensim.models.lsimodel:topic #1(553.568): -0.346*"fox" + -0.342*"lazy" + -0.342*

INFO:gensim.models.lsimodel:topic #2(1184.222): -0.792*"y" + -0.439*"n" + -0.185*"percent" + -0.113*"1" + -0.105*"republicans" + -0.101*"democrats" + -0.068*"million" + -0.066*"2" + -0.054*"x" + 0.052*"people"
INFO:gensim.models.lsimodel:topic #3(869.029): -0.660*"percent" + 0.278*"1" + 0.210*"cents" + 0.203*"million" + -0.140*"bush" + 0.138*"new" + 0.136*"cent" + 0.125*"lower" + 0.117*"futures" + 0.113*"higher"
INFO:gensim.models.lsimodel:topic #4(797.052): -0.427*"bush" + 0.252*"police" + -0.242*"president" + -0.195*"house" + 0.191*"people" + 0.166*"000" + -0.156*"d" + -0.151*"billion" + 0.147*"percent" + -0.120*"senate"
INFO:gensim.models.lsimodel:preparing a new chunk of documents
INFO:gensim.models.lsimodel:using 100 extra samples and 2 power iterations
INFO:gensim.models.lsimodel:1st phase: constructing (267318, 200) action matrix
INFO:gensim.models.lsimodel:orthonormalizing (267318, 200) action matrix
INFO:gensim.models.lsimodel:2nd phase: running dense svd on (200, 20000) matri

In [35]:
DOC_lsi = lsi_train[corpus]
QUERY_lsi = lsi_train[query_bow]
#save model to disk
lsi_train.save('models/lsi_train100')

INFO:gensim.utils:saving Projection object under models/lsi_train100.projection, separately None
INFO:gensim.utils:storing np array 'u' to models/lsi_train100.projection.u.npy
INFO:gensim.utils:saved models/lsi_train100.projection
INFO:gensim.utils:saving LsiModel object under models/lsi_train100, separately None
INFO:gensim.utils:not storing attribute projection
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:saved models/lsi_train100


In [37]:
#load lsi from disk
lsi_train = gensim.models.LsiModel.load('models/lsi_train100')

INFO:gensim.utils:loading LsiModel object from models/lsi_train100
INFO:gensim.utils:loading id2word recursively from models/lsi_train100.id2word.* with mmap=None
INFO:gensim.utils:setting ignored attribute projection to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loaded models/lsi_train100
INFO:gensim.utils:loading LsiModel object from models/lsi_train100.projection
INFO:gensim.utils:loading u from models/lsi_train100.projection.u.npy with mmap=None
INFO:gensim.utils:loaded models/lsi_train100.projection


In [43]:
def convert2vec(representation):
    vector = np.zeros(lsi_train.num_topics, dtype=float)
    for element in representation:
        vector[element[0]] = element[1]
    return vector

In [63]:
#create query and document representation

document_representation = {}
lsi_cosine_similarity = {}

for doc_lsi_term, doc_lsi_id in (zip(DOC_lsi, range(num_documents))):
    if(doc_lsi_id%5000==0):
        print('Doc processed is: ', str(doc_lsi_id))
    document_representation[doc_lsi_id] = doc_lsi_term

Doc processed is:  0
Doc processed is:  5000
Doc processed is:  10000
Doc processed is:  15000
Doc processed is:  20000
Doc processed is:  25000
Doc processed is:  30000
Doc processed is:  35000
Doc processed is:  40000
Doc processed is:  45000
Doc processed is:  50000
Doc processed is:  55000
Doc processed is:  60000
Doc processed is:  65000
Doc processed is:  70000
Doc processed is:  75000
Doc processed is:  80000
Doc processed is:  85000
Doc processed is:  90000
Doc processed is:  95000
Doc processed is:  100000
Doc processed is:  105000
Doc processed is:  110000
Doc processed is:  115000
Doc processed is:  120000
Doc processed is:  125000
Doc processed is:  130000
Doc processed is:  135000
Doc processed is:  140000
Doc processed is:  145000
Doc processed is:  150000
Doc processed is:  155000
Doc processed is:  160000


In [67]:
for query_lsi_id, query_lsi_term in (zip(tokenized_queries.items(), QUERY_lsi)):
    print('Query is: ', str(query_lsi_id))
    lsi_cosine_similarity[query_lsi_id[0]] = []
    query_lsi_representation = convert2vec(query_lsi_term)
    
    for result in top1000_tfidf[query_lsi_id[0]]:
        d_id = result[2]
        d = document_representation[d_id]
        document_lsi_representation = convert2vec(d)
        lsi_cosine_similarity[query_lsi_id[0]].append((cosine_similarity(query_lsi_representation.reshape(1,-1), document_lsi_representation.reshape(1, -1)), index.document(d_id)))

Query is:  ('84', [2563, 201208, 601, 351, 851, 122153])
Query is:  ('66', [1535, 1434, 3628])
Query is:  ('98', [91954, 176407, 851, 2324])
Query is:  ('161', [4072, 654])
Query is:  ('168', [2387, 23989])
Query is:  ('195', [75, 57, 29891, 774, 121675, 235])
Query is:  ('80', [214, 252, 789, 187515])
Query is:  ('64', [1987, 451])
Query is:  ('51', [5872, 3066])
Query is:  ('58', [3454, 1920])
Query is:  ('55', [4400, 235])
Query is:  ('121', [168, 1042])
Query is:  ('147', [5442, 5609, 5465, 276, 395])
Query is:  ('94', [774, 20692, 1033])


KeyError: 164597

In [ ]:
for query_lsi_id, query_lsi_term in (zip(tokenized_queries.items(), QUERY_lsi)):
    pprint(lsi_cosine_similarity[query_lsi_id[0]])

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

#### WORD2VEC

In [193]:
#creating names, constants and files
word2vec_file = 'word2vec_model.bin'
doc2vec_file = 'doc2vec_model.doc2vec'

corpus_file = 'pyndri_corpus.mm'
dict_file='corpdict.dict'

tfidf_model_file = 'tfidf_model.model'
tfidf_corpus_file = 'tfidf_corpus.mm'

lsi_model_file = 'lsi_model.model'
lsi_corpus_file = 'lsi_corpus.mm'

lda_model_file = 'lda_model.model'
lda_corpus_file = 'lda_corpus.mm'

LSI_SHAPE = 64
LDA_SHAPE = 32
NN_LAYERS = 32

AVAILABLE_CORES = multiprocessing.cpu_count()

logging.info("Created variables")

INFO:root:Created variables


In [194]:
logging.info('Initializing word2vec.')

dictionary = pyndri.extract_dictionary(index)
sentences = pyndri.compat.IndriSentences(index, dictionary)

if not os.path.isfile(word2vec_file):
    #train a model if it is not present
    model = gensim.models.Word2Vec(sentences,
                                  size = NN_LAYERS,
                                  window = 5, # One-sided window size
                                  min_count = 5, # Minimum word frequency
                                  workers = AVAILABLE_CORES,
                                  sample = 1e-5, # Sub-sample threshold
                                  negative = 5, # Number of negative examples
                                  sg = True)
    model.wv.save_word2vec_format(word2vec_file, binary = True)
    
else:
    #load existing model
    word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, binary = True)

INFO:root:Initializing word2vec.
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 2607270 words, keeping 73866 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 5208413 words, keeping 101162 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 7779447 words, keeping 121185 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 10402346 words, keeping 138774 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 12981963 words, keeping 153153 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 15578280 words, keeping 165943 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 18181273 words, keeping 177185 word types
INFO:gensim.models.word2vec:PROGRESS: at 

INFO:gensim.models.word2vec:PROGRESS: at 9.80% examples, 157824 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 9.98% examples, 157992 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 10.15% examples, 157879 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 10.34% examples, 157885 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 10.51% examples, 157886 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 10.70% examples, 157963 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 10.87% examples, 157864 words/s, in_qsize 8, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 11.05% examples, 157870 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 11.22% examples, 157815 words/s, in_qsize 7, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 11.41% examples, 157908 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.wor

INFO:gensim.models.word2vec:PROGRESS: at 24.78% examples, 157728 words/s, in_qsize 0, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 24.95% examples, 157802 words/s, in_qsize 0, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 25.13% examples, 157819 words/s, in_qsize 0, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 25.28% examples, 157645 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 25.46% examples, 157666 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 25.64% examples, 157709 words/s, in_qsize 4, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 25.81% examples, 157653 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 25.99% examples, 157661 words/s, in_qsize 0, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 26.15% examples, 157537 words/s, in_qsize 4, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 26.30% examples, 157420 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.w

INFO:gensim.models.word2vec:PROGRESS: at 40.38% examples, 160190 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 40.60% examples, 160383 words/s, in_qsize 1, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 40.79% examples, 160403 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 40.98% examples, 160476 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 41.16% examples, 160505 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 41.35% examples, 160549 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 41.54% examples, 160596 words/s, in_qsize 6, out_qsize 2
INFO:gensim.models.word2vec:PROGRESS: at 41.74% examples, 160627 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 41.94% examples, 160685 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 42.15% examples, 160807 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.w

INFO:gensim.models.word2vec:PROGRESS: at 56.59% examples, 163322 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 56.79% examples, 163367 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 56.98% examples, 163390 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 57.16% examples, 163394 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 57.37% examples, 163424 words/s, in_qsize 7, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 57.57% examples, 163446 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 57.76% examples, 163464 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 57.96% examples, 163501 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 58.17% examples, 163514 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 58.38% examples, 163588 words/s, in_qsize 5, out_qsize 1
INFO:gensim.models.w

INFO:gensim.models.word2vec:PROGRESS: at 72.80% examples, 165203 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 72.99% examples, 165209 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 73.19% examples, 165241 words/s, in_qsize 4, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 73.40% examples, 165259 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 73.61% examples, 165288 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 73.80% examples, 165296 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 74.00% examples, 165334 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 74.21% examples, 165366 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 74.41% examples, 165386 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 74.60% examples, 165375 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.w

INFO:gensim.models.word2vec:PROGRESS: at 89.01% examples, 166402 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 89.21% examples, 166425 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 89.40% examples, 166443 words/s, in_qsize 5, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 89.61% examples, 166485 words/s, in_qsize 4, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 89.79% examples, 166472 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 89.99% examples, 166500 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.word2vec:PROGRESS: at 90.18% examples, 166507 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 90.38% examples, 166508 words/s, in_qsize 5, out_qsize 2
INFO:gensim.models.word2vec:PROGRESS: at 90.58% examples, 166539 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 90.77% examples, 166549 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.w

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 5: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
